In [2]:
# This is how I applied a binary mask on the images then from the build the dataset 
# Bad coding practice I know but just comment out training once your done generating the mask and move on to uncommenting test and so on

import cv2
import os

# For training masking 
input_dir = 'train/images/'
output_dir = 'Annotated/train'

#For test masking
#input_dir = 'test/images/'
#output_dir = 'Annotated/test'

#For valid masking
#input_dir = '/images/'
#output_dir = 'Annotated/valid'


# Create the output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Process each image in the input directory
for filename in os.listdir(input_dir):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        # Full path to the input image
        image_path = os.path.join(input_dir, filename)

        # Read the image
        image = cv2.imread(image_path)

        # Convert to grayscale
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Apply thresholding to create a binary mask
        _, binary_mask = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)

        # Create the output file path
        output_path = os.path.join(output_dir, filename)

        # Save the binary mask to the output directory
        cv2.imwrite(output_path, binary_mask)



In [3]:
import os

import cv2

# Labels for train
input_dir = 'Annotated/train'
output_dir = 'train/labels'

# Labels for test
#input_dir = 'Annotated/test'
#output_dir = 'test/labels'

# Labels for valid
#input_dir = 'Annotated/valid'
#output_dir = 'valid/labels'

for j in os.listdir(input_dir):
    image_path = os.path.join(input_dir, j)
    # load the binary mask and get its contours
    mask = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    _, mask = cv2.threshold(mask, 1, 255, cv2.THRESH_BINARY)

    H, W = mask.shape
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # convert the contours to polygons
    polygons = []
    for cnt in contours:
        if cv2.contourArea(cnt) > 200:
            polygon = []
            for point in cnt:
                x, y = point[0]
                polygon.append(x / W)
                polygon.append(y / H)
            polygons.append(polygon)

    # print the polygons
    with open('{}.txt'.format(os.path.join(output_dir, j)[:-4]), 'w') as f:
        for polygon in polygons:
            for p_, p in enumerate(polygon):
                if p_ == len(polygon) - 1:
                    f.write('{}\n'.format(p))
                elif p_ == 0:
                    f.write('0 {} '.format(p))
                else:
                    f.write('{} '.format(p))

        f.close()